In [1]:
from utils.ModelWrapper import ModelWrapper
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
import wandb

In [2]:
# Init
data_dir = "./data/data-large/data"
img_size = 128

In [3]:
# Helper class
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

### Wandb Sweep training of the model

In [4]:

config_defaults = {
      'epochs': 20,
      'batch_size': 32,
      'learning_rate': 0.0001,
      'optimizer': 'nadam',
      'hidden_layer_size': 64,
      'conv_layer_1_size': 16,
      'conv_layer_2_size': 32,
      'conv_layer_3_size': 64,
      'dropout': 0.5,
   }

# # Config is a variable that holds and saves hyperparameters and inputs
config = dotdict(config_defaults)

# init wrapper
model_wrapper = ModelWrapper(data_dir, img_size, config.batch_size)

input_layers = [
   layers.Conv2D(filters = config.conv_layer_1_size, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(img_size, img_size, 3)),
   layers.MaxPooling2D(),
   layers.Conv2D(filters = config.conv_layer_2_size, kernel_size=(3, 3), padding='same', activation='relu'),
   layers.MaxPooling2D(),
   layers.Conv2D(filters = config.conv_layer_3_size, kernel_size=(3, 3), padding='same', activation='relu'),
   layers.MaxPooling2D(),
   layers.Dropout(config.dropout) if config.dropout > 0 else None,
   layers.Flatten(),
   layers.Dense(config.hidden_layer_size, activation='relu'),
   layers.Dense(units = 1, activation='sigmoid')
]
input_layers = [e for e in input_layers if e is not None]

# create model
model_wrapper.create_model(None, input_layers, config, use_wandb=False)

# Define the optimizer
if config.optimizer=='sgd':
   optimizer = tf.keras.optimizers.SGD(lr=config.learning_rate) 
elif config.optimizer=='rmsprop':
   optimizer = tf.keras.optimizers.RMSprop(lr=config.learning_rate,)
elif config.optimizer=='adam':
   optimizer = tf.keras.optimizers.Adam(lr=config.learning_rate,) 
elif config.optimizer=='nadam':
   optimizer = tf.keras.optimizers.Nadam(lr=config.learning_rate,) 

# Compile the model
model_wrapper.model.compile(optimizer=optimizer,loss='binary_crossentropy', metrics=['accuracy'])
# Train the model
model_wrapper.fit()

Found 29988 images belonging to 2 classes.
Found 3431 images belonging to 2 classes.
Found 1587 images belonging to 2 classes.


c:\Users\henriks\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizers\optimizer_v2\nadam.py:78: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Nadam, self).__init__(name, **kwargs)
c:\Users\henriks\Documents\skole\master_linje\2_semester\DAT550\prosjekt\DAT550-project\utils\ModelWrapper.py:102: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  return self.model.fit_generator(


Epoch 1/20
 14/938 [..............................] - ETA: 5:57 - loss: 0.6922 - accuracy: 0.5246

### Evaluate model

In [ ]:
# load the bess model NB: edit namd and path
loaded_model = wandb.restore('model-best.h5', run_path="dat550/deepfake-efficientnet/runs/29425ckm")
best_model = load_model(loaded_model.name)

In [ ]:
model_wrapper = ModelWrapper(data_dir, img_size, 32)
model_wrapper.evaluate_model(best_model)

In [ ]:
model_wrapper.export_to_png()